<a href="https://colab.research.google.com/github/PushkarShah743125/HackInverse-0.5/blob/main/HackInverse_0_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- GOOGLE DRIVE MOUNTING ---
from google.colab import drive
drive.mount('/content/drive')
# --- SECURE DATABASE INITIALIZATION ---
import sqlite3
import os
import hashlib
import secrets
DB_FILE = 'secure_admin.db'
def hash_password(password, salt=None):
    if not salt:
        salt = secrets.token_hex(16)
    # 100,000 iterations for robust security
    pwd_hash = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'),
                                   salt.encode('utf-8'), 100000).hex()
    return pwd_hash, salt
def create_initial_database():
    if os.path.exists(DB_FILE):
        os.remove(DB_FILE)
        print(f"Removed existing corrupted database file: {DB_FILE}")
    print("Initializing secure database...")
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute('''
CREATE TABLE IF NOT EXISTS admin_users (
id INTEGER PRIMARY KEY AUTOINCREMENT,
username TEXT UNIQUE NOT NULL,
password_hash TEXT NOT NULL,
salt TEXT NOT NULL
)
''')
    p_hash, p_salt = hash_password("SuperSecure123!")
    try:
        cursor.execute("INSERT INTO admin_users (username, password_hash, salt) VALUES (?, ?, ?)",
                       ("admin", p_hash, p_salt))
        conn.commit()
        print("Database 'secure_admin.db' created and 'admin' user added successfully.")
    except sqlite3.IntegrityError:
        print("Admin user already exists.")
    conn.close()
create_initial_database()

Mounted at /content/drive
Initializing secure database...
Database 'secure_admin.db' created and 'admin' user added successfully.


In [ ]:
import sqlite3
import pandas as pd
import os
DB_FILE = 'secure_admin.db'
# --- DATA INSPECTION ---
if os.path.exists(DB_FILE):
    conn = sqlite3.connect(DB_FILE)
    df_admin_users = pd.read_sql_query("SELECT * FROM admin_users", conn)
    print("Contents of 'admin_users' table:")
    display(df_admin_users)
    conn.close()
else:
    print(f"Database file '{DB_FILE}' not found.")
# --- SCHEMA INSPECTION ---
if os.path.exists(DB_FILE):
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print(f"Tables in {DB_FILE}:")
    for table in tables:
        print(f"- {table[0]}")
    conn.close()

Contents of 'admin_users' table:


,id,username,password_hash,salt
0,1,admin,1566cf122d6b85076760515e78d951e47410a62cfacb54...,5a49ca0e4624200c58a0fff768b576b9


Tables in secure_admin.db:
- admin_users
- sqlite_sequence


In [ ]:
# ATTACK SCRIPT
filename = "secure_admin.db"
with open(filename, "a") as f:
    f.write("HACKED_DATA_INJECTION_X99")
print(f"Attack successful: Corrupted {filename}")

Attack successful: Corrupted secure_admin.db


In [ ]:
import os
import time
import json
import shutil
import sqlite3
import hashlib
import logging
import secrets
from datetime import datetime

# ================= CONFIG =================
class Config:
    DB_FILE = "secure_admin.db"
    LOCAL_BACKUP = "secure_admin.db.bak"
    AUDIT_LOG_FILE = "security_audit_report.jsonl"  # JSON lines = faster
    CHECK_INTERVAL = 3

    CLOUD_BACKUP_DIR = "/content/drive/My Drive/Secure_Backups/"
    CLOUD_BACKUP_FILE = os.path.join(CLOUD_BACKUP_DIR, "secure_admin_backup.db")


# ================= LOGGING =================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - [SENTINEL] - %(message)s"
)


def log_security_event(event_type, details, severity="MEDIUM"):
    """Append-only JSON logging (fast)."""
    entry = {
        "timestamp": datetime.now().isoformat(),
        "severity": severity,
        "event_type": event_type,
        "details": details,
        "file_target": Config.DB_FILE,
    }
    with open(Config.AUDIT_LOG_FILE, "a") as f:
        f.write(json.dumps(entry) + "\n")


# ================= DATABASE =================
def hash_password(password, salt=None):
    salt = salt or secrets.token_hex(16)
    pwd_hash = hashlib.pbkdf2_hmac(
        "sha256", password.encode(), salt.encode(), 100000
    ).hex()
    return pwd_hash, salt


def create_initial_database():
    # Always ensure a clean start for the sentinel by removing any existing file
    if os.path.exists(Config.DB_FILE):
        os.remove(Config.DB_FILE)
        logging.info(f"Removed existing database file: {Config.DB_FILE} for clean initialization.")

    logging.info("Initializing secure database...")
    with sqlite3.connect(Config.DB_FILE) as conn:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS admin_users (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                username TEXT UNIQUE NOT NULL,
                password_hash TEXT NOT NULL,
                salt TEXT NOT NULL
            )
        """)
        p_hash, p_salt = hash_password("SuperSecure123!")
        try:
            cursor.execute(
                "INSERT INTO admin_users VALUES (NULL, ?, ?, ?)",
                ("admin", p_hash, p_salt)
            )
        except sqlite3.IntegrityError:
            pass


# ================= HASHING =================
def calculate_hash(filepath):
    if not os.path.exists(filepath):
        return None

    sha256_hash = hashlib.sha256()
    with open(filepath, "rb") as f:
        for block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(block)
    return sha256_hash.hexdigest()


# ================= BACKUP SYSTEM =================
class BackupManager:
    def upload_to_drive(self):
        if not os.path.exists(Config.CLOUD_BACKUP_DIR):
            os.makedirs(Config.CLOUD_BACKUP_DIR)

        try:
            shutil.copy2(Config.DB_FILE, Config.CLOUD_BACKUP_FILE)
            logging.info("Backup synced to Google Drive.")
        except Exception as e:
            logging.error(f"Drive backup failed: {e}")

    def restore_from_drive(self):
        try:
            shutil.copy2(Config.CLOUD_BACKUP_FILE, Config.DB_FILE)
            logging.info("Restored from cloud backup.")
            return True
        except Exception:
            return False


# ================= SENTINEL =================
class Sentinel:
    def __init__(self):
        self.backup = BackupManager()
        create_initial_database()
        shutil.copy2(Config.DB_FILE, Config.LOCAL_BACKUP)
        self.backup.upload_to_drive()
        self.expected_hash = calculate_hash(Config.DB_FILE)

    def heal_system(self, incident):
        logging.warning(f"SECURITY INCIDENT: {incident}")
        log_security_event("INTEGRITY_BREACH", incident, "CRITICAL")

        if not self.backup.restore_from_drive():
            shutil.copy2(Config.LOCAL_BACKUP, Config.DB_FILE)

        self.expected_hash = calculate_hash(Config.DB_FILE)

    def run(self):
        logging.info(f"Watching {Config.DB_FILE}")
        while True:
            try:
                time.sleep(Config.CHECK_INTERVAL)
                current_hash = calculate_hash(Config.DB_FILE)

                if current_hash is None:
                    self.heal_system("FILE_DELETED")
                elif current_hash != self.expected_hash:
                    self.heal_system("UNAUTHORIZED_MODIFICATION")

            except KeyboardInterrupt:
                logging.info("Sentinel stopped.")
                break
            except Exception as e:
                logging.error(f"Unexpected error: {e}")


if __name__ == "__main__":
    Sentinel().run()

In [ ]:
import json
import datetime
import os
LOG_FILE = "security_audit_report.json"
def analyze_logs():
    print("--- CYBERSECURITY INCIDENT REPORT ---")
    print(f"Source: {LOG_FILE}")
    print(f"Generated: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("-" * 40)
    # ... (Log parsing logic, refer to full code in Cell 7)
    # Let's assume the log parsing logic from Cell 7 is defined here for a complete fix
    total_events = 0
    high_severity_count = 0
    threats_neutralized = 0
    attackers = {}

    if os.path.exists(LOG_FILE):
        try:
            with open(LOG_FILE, 'r') as f:
                logs = json.load(f)
            total_events = len(logs)
            for entry in logs:
                if entry.get("severity") == "CRITICAL":
                    high_severity_count += 1
                if entry.get("event_type") == "INTEGRITY_BREACH":
                    threats_neutralized += 1
                    file_target = entry.get("file_target", "UNKNOWN")
                    attackers[file_target] = attackers.get(file_target, 0) + 1
        except json.JSONDecodeError:
            print(f"Error: Corrupted JSON log file {LOG_FILE}")
            logs = []
    else:
        print(f"Log file '{LOG_FILE}' not found. No incidents to report.")
        return

    try:
        # ... (Counting logic)
        print(f"Total Log Entries Scanned: {total_events}")
        print(f"High Severity Alerts: {high_severity_count}")
        print(f"Active Threats Neutralized: {threats_neutralized}")
        print("-" * 40)
        if threats_neutralized > 0:
            print("TOP ATTACK VECTORS (File Targets/Incidents):")
            for proc, count in attackers.items():
                print(f" [x] {proc}: {count} times")
        else:
            print("No active threats detected in logs.")
        print("-" * 40)
        print("STATUS: SYSTEM SECURE")
    except Exception as e:
        print(f"Critical Error analyzing logs: {e}")

if __name__ == "__main__":
    analyze_logs()

--- CYBERSECURITY INCIDENT REPORT ---
Source: security_audit_report.json
Generated: 2026-01-31 05:49:47
----------------------------------------
Total Log Entries Scanned: 20
High Severity Alerts: 20
Active Threats Neutralized: 20
----------------------------------------
TOP ATTACK VECTORS (File Targets/Incidents):
 [x] secure_admin.db: 20 times
----------------------------------------
STATUS: SYSTEM SECURE


In [ ]:
import sqlite3
import os
# CONFIGURATION
TARGET_FILE = "secure_admin.db"
def simulate_unauthorized_modification():
    if not os.path.exists(TARGET_FILE):
        print(f"Error: Target file '{TARGET_FILE}' not found.")
        return
    print(f"Connecting to {TARGET_FILE}...")
    try:
        # Connect to the database
        conn = sqlite3.connect(TARGET_FILE)
        cursor = conn.cursor()
        # Check existing data before modification
        print("\n--- BEFORE TAMPERING ---")
        cursor.execute("SELECT * FROM admin_users")
        rows = cursor.fetchall()
        for row in rows:
            print(row)
        # SIMULATE THE ATTACK: Modify the username of the admin
        print("\n[!] Attempting to modify admin username to 'HACKED_ADMIN'...")
        cursor.execute("UPDATE admin_users SET username = ? WHERE username = ?",
                       ("HACKED_ADMIN", "admin"))
        if cursor.rowcount == 0:
            print("Target user 'admin' not found. Creating a rogue user instead...")
            cursor.execute("""INSERT INTO admin_users (username, password_hash, salt)
                           VALUES (?, ?, ?)""",
                           ("rogue_user", "12345", "random_salt"))
            conn.commit()
            print("SUCCESS: Database modified. Commit executed.")
        # Check data after modification
        print("\n--- AFTER TAMPERING ---")
        cursor.execute("SELECT * FROM admin_users")
        rows = cursor.fetchall()
        for row in rows:
            print(row)
        conn.close()
        print("\n[!] Integrity Compromised. The file hash has changed.")
    except sqlite3.Error as e:
        print(f"Database Error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
if __name__ == "__main__":
    simulate_unauthorized_modification()

Connecting to secure_admin.db...

--- BEFORE TAMPERING ---
(1, 'admin', '1566cf122d6b85076760515e78d951e47410a62cfacb546b6529e8dd657dfffc', '5a49ca0e4624200c58a0fff768b576b9')

[!] Attempting to modify admin username to 'HACKED_ADMIN'...

--- AFTER TAMPERING ---
(1, 'HACKED_ADMIN', '1566cf122d6b85076760515e78d951e47410a62cfacb546b6529e8dd657dfffc', '5a49ca0e4624200c58a0fff768b576b9')

[!] Integrity Compromised. The file hash has changed.


In [ ]:
import json
import datetime
import os
# --- CONFIGURATION ---
LOG_FILE = "security_audit_report.json" # Changed to match the actual audit log file
def analyze_logs():
    print("--- CYBERSECURITY INCIDENT REPORT ---")
    print(f"Source: {LOG_FILE}")
    print(f"Generated: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("-" * 40)
    if not os.path.exists(LOG_FILE):
        print(f"ERROR: Log file '{LOG_FILE}' not found.")
        print("Run the sentry script and trigger an event first.")
        return
    # Statistics Counters
    total_events = 0
    threats_neutralized = 0
    high_severity_count = 0
    attackers = {} # To track distinct process names
    try:
        with open(LOG_FILE, 'r') as f:
            try:
                all_entries = json.load(f)
            except json.JSONDecodeError:
                print(f"WARNING: Log file '{LOG_FILE}' contains malformed JSON or is empty.")
                # Create an empty list if the file is malformed or empty to avoid further errors
                all_entries = []
        if not isinstance(all_entries, list):
            print(f"WARNING: Log file '{LOG_FILE}' does not contain a JSON array. Found type: {type(all_entries)}")
            all_entries = [] # Reset to empty to prevent iterating over non-list
        for entry in all_entries:
            total_events += 1
            # extract data
            severity = entry.get('severity', 'UNKNOWN')
            # Logic: Count High Severity / Kills
            if severity == 'CRITICAL':
                high_severity_count += 1
            # The log_security_event logs 'INTEGRITY_BREACH' events
            if entry.get('event_type') == 'INTEGRITY_BREACH':
                threats_neutralized += 1
                # For integrity breaches, 'file_target' is a top-level key
                attacker_id = entry.get('file_target', 'unknown_attacker')
                if attacker_id in attackers:
                    attackers[attacker_id] += 1
                else:
                    attackers[attacker_id] = 1
        # --- PRINT THE REPORT ---
        print(f"Total Log Entries Scanned: {total_events}")
        print(f"High Severity Alerts: {high_severity_count}")
        print(f"Active Threats Neutralized: {threats_neutralized}")
        print("-" * 40)
        if threats_neutralized > 0:
            print("TOP ATTACK VECTORS (File Targets/Incidents):")
            for proc, count in attackers.items():
                print(f" [x] {proc}: {count} times")
        else:
            print("No active threats detected in logs.")
        print("-" * 40)
        print("STATUS: SYSTEM SECURE")
    except Exception as e:
        print(f"Critical Error analyzing logs: {e}")
if __name__ == "__main__":
    analyze_logs()

--- CYBERSECURITY INCIDENT REPORT ---
Source: security_audit_report.json
Generated: 2026-01-31 05:49:47
----------------------------------------
Total Log Entries Scanned: 20
High Severity Alerts: 20
Active Threats Neutralized: 20
----------------------------------------
TOP ATTACK VECTORS (File Targets/Incidents):
 [x] secure_admin.db: 20 times
----------------------------------------
STATUS: SYSTEM SECURE
